In [1]:
# %%
import pandas as pd
#import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 

In [2]:
# %%
train = pd.read_csv('./data/train.csv')
test =  pd.read_csv('./data/test.csv')
trade = pd.read_csv('./data/international_trade.csv')

train.head()


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [3]:
# %%
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

In [4]:
# %%
import datetime
import holidays

# %%
#train = train[(train['supply(kg)'] != 0) & (train['supply(kg)'] != 0)] # 이거차이도 큼!!! 거의 rmse 1200에서 680으로 줄어듦. 근데 이건 validation에만이고 test는 이거하면 정반대로됨 ㅠ...

# %%
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# %%
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [5]:
# %%
from sklearn.preprocessing import LabelEncoder
train_x = train.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)','Date'])
train_y = train['price(원/kg)']
test_x = test_re.drop(columns=['ID', 'timestamp','Date'])
qual_col = ['item', 'corporation', 'location','day_name']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

In [6]:
train_x = pd.concat([train_x, train_y], axis=1)

In [7]:
train_x.head()

,item,corporation,location,week,day_name,year,month,day,price(원/kg)
0,4,0,0,1,5,2019,1,1,0.0
1,4,0,0,1,6,2019,1,2,0.0
2,4,0,0,1,4,2019,1,3,1728.0
3,4,0,0,1,0,2019,1,4,1408.0
4,4,0,0,1,2,2019,1,5,1250.0


In [8]:
test_x.head() #day_name 일치

,item,corporation,location,year,month,day,week,day_name
0,4,0,0,2023,3,4,9,2
1,4,0,0,2023,3,5,9,3
2,4,0,0,2023,3,6,10,1
3,4,0,0,2023,3,7,10,5
4,4,0,0,2023,3,8,10,6


In [9]:
# import pycaret regression and init setup
from pycaret.regression import *

s = setup(train_x, target = 'price(원/kg)', 
        session_id = 41, 
        fold = 10, 
        normalize = True, 
        normalize_method = 'minmax', 
        train_size=0.9)

# %%
best_model = compare_models()


,Description,Value
0,Session id,41
1,Target,price(원/kg)
2,Target type,Regression
3,Original data shape,"(59397, 9)"
4,Transformed data shape,"(59397, 9)"
5,Transformed train set shape,"(53457, 9)"
6,Transformed test set shape,"(5940, 9)"
7,Numeric features,8
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,321.8376,777283.7005,880.4287,0.8121,2.2299,0.2004,1.1390
xgboost,Extreme Gradient Boosting,424.1630,793199.6188,890.0267,0.8082,3.7720,0.2513,0.0940
catboost,CatBoost Regressor,438.6402,811286.9653,900.1742,0.8038,3.8278,0.2532,1.1050
lightgbm,Light Gradient Boosting Machine,432.6489,823144.9651,906.8225,0.8010,3.7147,0.2684,0.4880
et,Extra Trees Regressor,331.0759,828929.5876,909.3142,0.7997,2.2538,0.2081,0.8540
knn,K Neighbors Regressor,532.1389,1182276.8750,1086.9761,0.7141,3.0491,0.3101,0.0520
gbr,Gradient Boosting Regressor,649.8427,1262389.1945,1123.3304,0.6947,4.3683,0.3335,0.6800
dt,Decision Tree Regressor,390.7192,1444553.6005,1199.7742,0.6508,2.1238,0.2716,0.0410
ada,AdaBoost Regressor,1134.1871,2450928.5624,1564.7586,0.4072,5.2065,0.5066,0.3810
ridge,Ridge Regression,1330.2471,3544529.3426,1882.5394,0.1434,5.1347,0.9098,0.0170


In [10]:
llar  = create_model('rf')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,333.4317,833722.3100,913.0840,0.7976,2.2219,0.1950
1,316.7226,764802.6103,874.5299,0.8108,2.2449,0.1941
2,330.6884,861436.8185,928.1362,0.7863,2.2599,0.2047
3,334.2406,845042.4334,919.2619,0.7998,2.2588,0.2067
4,324.6114,762771.6131,873.3680,0.8193,2.2473,0.1986
5,335.2364,855842.3159,925.1175,0.8045,2.2122,0.1982
6,306.4809,678254.0721,823.5618,0.8377,2.2298,0.1997
7,287.7168,603293.8776,776.7199,0.8516,2.1586,0.1929
8,326.9675,806282.5553,897.9324,0.7987,2.2361,0.2108


In [12]:
tuned_clf = tune_model(llar, optimize = 'RMSE')

predictions = predict_model(tuned_clf, data = test_x)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,21:38:50
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


KeyboardInterrupt: 

In [ ]:
result = pd.concat([test_re, predictions['prediction_label']], axis = 1)

result['answer'] = result['prediction_label'].copy()

del result['timestamp']
del result['item']
del result['corporation']
del result['location']
del result['year']
del result['month']
del result['day']
del result['Date']
del result['week']
del result['day_name']
del result['prediction_label']

In [ ]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

result.head(20)

,ID,answer
0,TG_A_J_20230304,3775.64
1,TG_A_J_20230305,0.00
2,TG_A_J_20230306,3379.94
3,TG_A_J_20230307,3173.52
4,TG_A_J_20230308,3382.07
5,TG_A_J_20230309,3813.71
6,TG_A_J_20230310,3348.54
7,TG_A_J_20230311,3539.96
8,TG_A_J_20230312,0.00
9,TG_A_J_20230313,3655.10


In [ ]:
result.to_csv('result1_tuned.csv', index = False)